# 1. SOMTH
### 주로 Oversampling에 근간이 되는 SMOTE는 이름 그대로 minority class에서 synthetic 샘플을 생성하는 방법이다. k-nearest neighbors(knn)를 이용한다는 점이 가장 큰 특징으로 먼저 knn으로 가까운 minority class 들을 찾은 후 0과 1사이의 랜덤한 값을 선택한 후 해당 값으로 내분하는 점을 새로운 샘플로 생성하는 기법이다. 즉 minority 관측치들 사이의 linear 상에서 새로운 관측치가 생성된다. 이런 방식은 minority class에 변주를 주기 때문에 처음 지적했던 Oversampling의 단점을 보완하게 된다. 그림으로 간단하게 표현하면 다음과 같다.

In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("./데이터자료/9-2. Std 데이터셋.csv")
feature = pd.read_csv("./데이터자료/8-2. Std feature.csv")
df

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,label,회계년
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,1.186494,-0.286555,-0.242712,...,-0.517775,-0.268715,-0.204982,-0.097181,0.001823,0.167352,-0.034998,-0.128958,0.0,2011
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-0.465320,-0.266139,-0.145700,...,-0.570848,-0.965957,-0.239707,-0.097246,-0.455154,7.331845,-8.395971,-8.842764,0.0,2012
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.399443,-0.034331,0.368591,...,-0.459402,-0.280668,-0.160693,-0.097092,0.377872,-0.016097,0.127964,0.136844,0.0,2013
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.042989,0.633335,-0.160104,...,-0.458301,-0.300589,-0.202938,-0.097179,0.294901,0.063259,0.057470,0.125748,0.0,2014
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.229752,-0.280476,-0.110645,...,-0.503803,-0.558426,-0.217252,-0.097201,-0.090083,0.704439,-0.512105,-0.358349,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,휴마시스(주),205470,2020-12-01,2014-12-01,NaN,5,의료용 물질 및 의약품 제조업,5.662930,1.347455,2.510801,...,-0.151711,1.140567,-0.125778,-0.096931,0.930074,-0.576239,0.625551,0.569922,0.0,2020
9274,휴젤(주),145020,2019-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,0.059296,-0.046260,0.113775,...,1.299954,-0.272699,-0.062404,-0.097006,0.914756,-0.434150,0.499330,0.360994,0.0,2019
9275,휴젤(주),145020,2020-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,-0.170016,-0.100971,-0.437478,...,1.349171,-0.301727,-0.058107,-0.096969,0.907353,-0.415350,0.482630,0.330828,0.0,2020
9276,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-0.488761,-0.299172,0.793390,...,-0.392823,-0.500939,-0.145811,-0.095817,-0.354313,-0.106535,0.208302,0.265758,0.0,2019


In [4]:
feature

,mda_feature,logit_feature,lasso_feature,ela_feature,same_feature
0,자기자본구성비율,매출액증가율,총자본증가율,자기자본증가율,자기자본구성비율
1,매출액순이익률,유형자산증가율,자기자본순이익률,총자본증가율,자기자본배율
2,순운전자본비율,재고자산증가율,총자본사업이익률,매출액순이익률,총자본증가율
3,순운전자본회전률,비유동자산증가율,총자본순이익률,자기자본순이익률,매출채권회전기간
4,총자본투자효율,유동자산증가율,순운전자본비율,총자본사업이익률,총자본순이익률
5,자기자본순이익률,현금비율,유동자산구성비율,총자본순이익률,총자본사업이익률
6,총자본증가율,이자보상배율이자비용,자기자본구성비율,순운전자본비율,순운전자본비율
7,총자본사업이익률,비유동비율,자기자본배율,자기자본구성비율,순운전자본회전률
8,기계투자효율,자기자본구성비율,매출채권회전률,자기자본배율,자기자본순이익률
9,1주당순이익,자기자본배율,매출채권회전기간,매출채권회전기간,매출액순이익률


In [5]:
train = df[df["회계년"] <= 2018].reset_index(drop='index')
test  = df[df["회계년"] > 2018].reset_index(drop='index')
col = df.columns[7:55]

In [6]:
x_train = train[col]
y_train = train["label"]
x_test = test[col]
y_test = test["label"]

In [7]:
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(13584, 48)


0.0    6792
1.0    6792
Name: label, dtype: int64

In [8]:
mda_feature=feature["mda_feature"].tolist()
logit_feature=feature["logit_feature"].tolist()
lasso_feature=feature["lasso_feature"].tolist()
ela_feature=feature["ela_feature"].tolist()
same_feature=feature["same_feature"].tolist()

In [9]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [10]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")


In [11]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [12]:
mda_x_train = x_train[mda_feature].reset_index(drop='index')
mda_x_test = x_test[mda_feature].reset_index(drop='index')
mda_y_train = y_train
mda_y_test = test["label"]

In [13]:
logit_x_train = x_train[logit_feature].reset_index(drop='index')
logit_x_test = x_test[logit_feature].reset_index(drop='index')
logit_y_train = y_train
logit_y_test = test["label"]

In [14]:
lasso_x_train = x_train[lasso_feature].reset_index(drop='index')
lasso_x_test = x_test[lasso_feature].reset_index(drop='index')
lasso_y_train = y_train
lasso_y_test = test["label"]

In [15]:
ela_x_train = x_train[ela_feature].reset_index(drop='index')
ela_x_test = x_test[ela_feature].reset_index(drop='index')
ela_y_train = y_train
ela_y_test = test["label"]

In [16]:
same_x_train = x_train[same_feature].reset_index(drop='index')
same_x_test = x_test[same_feature].reset_index(drop='index')
same_y_train = y_train
same_y_test = test["label"]

In [17]:
fea_x_train = [mda_x_train, logit_x_train, lasso_x_train, ela_x_train, same_x_train]
fea_y_train = [mda_y_train, logit_y_train, lasso_y_train, ela_y_train, same_y_train]
fea_x_test = [mda_x_test, logit_x_test, lasso_x_test, ela_x_test, same_x_test]
fea_y_test = [mda_y_test, logit_y_test, lasso_y_test, ela_y_test, same_y_test]
fea = ["mda","logit","lasso","ela","same"]

In [18]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lda_clf.fit(x_train, y_train)
    lda_pred = lda_clf.predict(x_test)
    lda_pred_proba = lda_clf.predict_proba(x_test)
    print(feature,"lda") 
    get_clf_eval(y_test, lda_pred)
    get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)



mda lda
오차행렬
[[1888  427]
 [  15   35]]
정확도: 0.8131, 정밀도 : 0.0758, 재현율:0.7000,F1 스코어:0.1367
ROC 스코어: 0.7578, PR score : 0.0594
임곗값: 0.1
오차행렬
[[ 426 1889]
 [   4   46]]
정확도: 0.1996, 정밀도 : 0.0238, 재현율:0.9200,F1 스코어:0.0463
ROC 스코어: 0.5520, PR score : 0.0236
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[ 866 1449]
 [   9   41]]
정확도: 0.3835, 정밀도 : 0.0275, 재현율:0.8200,F1 스코어:0.0532
ROC 스코어: 0.5970, PR score : 0.0264
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1331  984]
 [  11   39]]
정확도: 0.5793, 정밀도 : 0.0381, 재현율:0.7800,F1 스코어:0.0727
ROC 스코어: 0.6775, PR score : 0.0344
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1653  662]
 [  13   37]]
정확도: 0.7146, 정밀도 : 0.0529, 재현율:0.7400,F1 스코어:0.0988
ROC 스코어: 0.7270, PR score : 0.0447
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1888  427]
 [  15   35]]
정확도: 0.8131, 정밀도 : 0.0758, 재현율:0.7000,F1 스코어:0.1367
ROC 스코어: 0.7578, PR score : 0.0594
-

In [19]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    dt_clf.fit(x_train, y_train)
    dt_pred = dt_clf.predict(x_test)
    dt_pred_proba = dt_clf.predict_proba(x_test)   
    print(feature,"디시전트리") 
    get_clf_eval(y_test, dt_pred)
    get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 디시전트리
오차행렬
[[2119  196]
 [  32   18]]
정확도: 0.9036, 정밀도 : 0.0841, 재현율:0.3600,F1 스코어:0.1364
ROC 스코어: 0.6377, PR score : 0.0438
임곗값: 0.1
오차행렬
[[2119  196]
 [  32   18]]
정확도: 0.9036, 정밀도 : 0.0841, 재현율:0.3600,F1 스코어:0.1364
ROC 스코어: 0.6377, PR score : 0.0438
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2119  196]
 [  32   18]]
정확도: 0.9036, 정밀도 : 0.0841, 재현율:0.3600,F1 스코어:0.1364
ROC 스코어: 0.6377, PR score : 0.0438
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2119  196]
 [  32   18]]
정확도: 0.9036, 정밀도 : 0.0841, 재현율:0.3600,F1 스코어:0.1364
ROC 스코어: 0.6377, PR score : 0.0438
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2119  196]
 [  32   18]]
정확도: 0.9036, 정밀도 : 0.0841, 재현율:0.3600,F1 스코어:0.1364
ROC 스코어: 0.6377, PR score : 0.0438
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2119  196]
 [  32   18]]
정확도: 0.9036, 정밀도 : 0.0841, 재현율:0.3600,F1 스코어:0.1364
ROC 스코어: 0.6377, PR score : 0.0438

In [20]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    # key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
    param_grid = {
      'criterion': ['entropy', 'gini'],
               'max_depth': [2,4,6,8],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4,6,8,10],
               'min_samples_split': [4,6,8,10]}
    grid_search = GridSearchCV(tree
                            , param_grid = param_grid
                            , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                            , n_jobs = -1)  #-1: 모든 코어 사용

    grid_search.fit(x_train, y_train)

    # 최선의 결과 출력
    print('피처:{2} 최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_, feature))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
    dt_clf_best.fit(x_train, y_train)
    dt_best_pred = dt_clf_best.predict(x_test)
    dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, dt_best_pred)
    get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


피처:mda 최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10}, 최적 평균 정확도 :0.870
mda 디시전트리 파라미터적용
오차행렬
[[1978  337]
 [  26   24]]
정확도: 0.8465, 정밀도 : 0.0665, 재현율:0.4800,F1 스코어:0.1168
ROC 스코어: 0.6672, PR score : 0.0429
임곗값: 0.1
오차행렬
[[ 804 1511]
 [  14   36]]
정확도: 0.3552, 정밀도 : 0.0233, 재현율:0.7200,F1 스코어:0.0451
ROC 스코어: 0.5337, PR score : 0.0227
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1796  519]
 [  21   29]]
정확도: 0.7717, 정밀도 : 0.0529, 재현율:0.5800,F1 스코어:0.0970
ROC 스코어: 0.6779, PR score : 0.0396
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1924  391]
 [  24   26]]
정확도: 0.8245, 정밀도 : 0.0624, 재현율:0.5200,F1 스코어:0.1113
ROC 스코어: 0.6756, PR score : 0.0426
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1947  368]
 [  24   26]]
정확도: 0.8342, 정밀도 : 0.0660, 재현율:0.5200,F1 스코어:0.1171
ROC 스코어: 0.6805, PR score : 0.0445
---------------------------

In [21]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    rf_clf.fit(x_train, y_train)
    rf_pred = rf_clf.predict(x_test)
    rf_pred_proba = rf_clf.predict_proba(x_test)
    print(feature,"랜덤포레스트") 
    get_clf_eval(y_test, rf_pred)
    get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 랜덤포레스트
오차행렬
[[2239   76]
 [  33   17]]
정확도: 0.9539, 정밀도 : 0.1828, 재현율:0.3400,F1 스코어:0.2378
ROC 스코어: 0.6536, PR score : 0.0761
임곗값: 0.1
오차행렬
[[1614  701]
 [  11   39]]
정확도: 0.6989, 정밀도 : 0.0527, 재현율:0.7800,F1 스코어:0.0987
ROC 스코어: 0.7386, PR score : 0.0458
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1947  368]
 [  22   28]]
정확도: 0.8351, 정밀도 : 0.0707, 재현율:0.5600,F1 스코어:0.1256
ROC 스코어: 0.7005, PR score : 0.0489
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2110  205]
 [  26   24]]
정확도: 0.9023, 정밀도 : 0.1048, 재현율:0.4800,F1 스코어:0.1720
ROC 스코어: 0.6957, PR score : 0.0613
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2193  122]
 [  28   22]]
정확도: 0.9366, 정밀도 : 0.1528, 재현율:0.4400,F1 스코어:0.2268
ROC 스코어: 0.6937, PR score : 0.0791
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2239   76]
 [  33   17]]
정확도: 0.9539, 정밀도 : 0.1828, 재현율:0.3400,F1 스코어:0.2378
ROC 스코어: 0.6536, PR score : 0.076

In [22]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    grid_search = {'criterion': ['entropy', 'gini'],
                'max_depth': [2,4,6,8],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [4,6,8,10],
                'min_samples_split': [4,6,8,10],
                'n_estimators': [20,40,60,80,100]}

    model = RandomForestClassifier(random_state=52)
    grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                                cv = 5,verbose=0, n_jobs = -1)
    grid_search.fit(x_train,y_train)

    # 최선의 결과 출력
    print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
    rf_clf_best.fit(x_train, y_train)
    rf_best_pred = rf_clf_best.predict(x_test)
    rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

    print(feature,"랜덤포레스트 파라미터적용") 
    get_clf_eval(y_test, rf_best_pred)
    get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 20}, 최적 평균 정확도 :0.890
mda 랜덤포레스트 파라미터적용
오차행렬
[[2085  230]
 [  24   26]]
정확도: 0.8926, 정밀도 : 0.1016, 재현율:0.5200,F1 스코어:0.1699
ROC 스코어: 0.7103, PR score : 0.0630
임곗값: 0.1
오차행렬
[[ 556 1759]
 [   4   46]]
정확도: 0.2545, 정밀도 : 0.0255, 재현율:0.9200,F1 스코어:0.0496
ROC 스코어: 0.5801, PR score : 0.0251
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1537  778]
 [  10   40]]
정확도: 0.6668, 정밀도 : 0.0489, 재현율:0.8000,F1 스코어:0.0922
ROC 스코어: 0.7320, PR score : 0.0433
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1832  483]
 [  14   36]]
정확도: 0.7899, 정밀도 : 0.0694, 재현율:0.7200,F1 스코어:0.1265
ROC 스코어: 0.7557, PR score : 0.0559
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1969  346]
 [  20   30]]
정확도: 0.8452, 정밀도 : 0.0798, 재현율:0.6000,F1 스코어:0.1408
ROC 스코어: 0.7253, PR score : 0.0563
----------------

In [23]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    lr_pred_proba = lr_clf.predict_proba(x_test)
    print(feature,"로지스틱회귀") 
    get_clf_eval(y_test, lr_pred)
    get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

mda 로지스틱회귀
오차행렬
[[1867  448]
 [  16   34]]
정확도: 0.8038, 정밀도 : 0.0705, 재현율:0.6800,F1 스코어:0.1278
ROC 스코어: 0.7432, PR score : 0.0547
임곗값: 0.1
오차행렬
[[ 476 1839]
 [   4   46]]
정확도: 0.2207, 정밀도 : 0.0244, 재현율:0.9200,F1 스코어:0.0475
ROC 스코어: 0.5628, PR score : 0.0241
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[ 886 1429]
 [   9   41]]
정확도: 0.3920, 정밀도 : 0.0279, 재현율:0.8200,F1 스코어:0.0539
ROC 스코어: 0.6014, PR score : 0.0267
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1314 1001]
 [  12   38]]
정확도: 0.5717, 정밀도 : 0.0366, 재현율:0.7600,F1 스코어:0.0698
ROC 스코어: 0.6638, PR score : 0.0329
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1637  678]
 [  14   36]]
정확도: 0.7074, 정밀도 : 0.0504, 재현율:0.7200,F1 스코어:0.0942
ROC 스코어: 0.7136, PR score : 0.0422
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1867  448]
 [  16   34]]
정확도: 0.8038, 정밀도 : 0.0705, 재현율:0.6800,F1 스코어:0.1278
ROC 스코어: 0.7432, PR score : 0.054

In [24]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {'penalty': ['l2','l1','elasticnet','None'],
            'C':[0.01,0.1,1,3,5,10]}

    grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
    grid_search.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
    lr_clf_best.fit(x_train, y_train)
    lr_best_pred = lr_clf_best.predict(x_test)
    lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

    print(feature,"로지스틱회귀 파라미터적용") 
    get_clf_eval(y_test, lr_best_pred)
    get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 0.01, 'penalty': 'l2'}, 최적 평균 정확도:0.793
mda 로지스틱회귀 파라미터적용
오차행렬
[[1873  442]
 [  16   34]]
정확도: 0.8063, 정밀도 : 0.0714, 재현율:0.6800,F1 스코어:0.1293
ROC 스코어: 0.7445, PR score : 0.0553
임곗값: 0.1
오차행렬
[[ 410 1905]
 [   4   46]]
정확도: 0.1928, 정밀도 : 0.0236, 재현율:0.9200,F1 스코어:0.0460
ROC 스코어: 0.5486, PR score : 0.0234
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[ 833 1482]
 [   8   42]]
정확도: 0.3700, 정밀도 : 0.0276, 재현율:0.8400,F1 스코어:0.0534
ROC 스코어: 0.5999, PR score : 0.0265
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1283 1032]
 [  11   39]]
정확도: 0.5590, 정밀도 : 0.0364, 재현율:0.7800,F1 스코어:0.0696
ROC 스코어: 0.6671, PR score : 0.0331
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1640  675]
 [  14   36]]
정확도: 0.7087, 정밀도 : 0.0506, 재현율:0.7200,F1 스코어:0.0946
ROC 스코어: 0.7142, PR score : 0.0424
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1873  442]
 [  16   34]]
정확도: 0.8063, 정밀도 : 0

In [25]:
# 1-5 KNN 
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    knn_clf.fit(x_train, y_train)
    knn_pred = knn_clf.predict(x_test)
    knn_pred_proba = knn_clf.predict_proba(x_test)
    print(feature,"KNN") 
    get_clf_eval(y_test, knn_pred)
    get_eval_by_threshold(y_test , knn_pred_proba[:,1].reshape(-1,1), thresholds)



mda KNN
오차행렬
[[2039  276]
 [  26   24]]
정확도: 0.8723, 정밀도 : 0.0800, 재현율:0.4800,F1 스코어:0.1371
ROC 스코어: 0.6804, PR score : 0.0494
임곗값: 0.1
오차행렬
[[1844  471]
 [  21   29]]
정확도: 0.7920, 정밀도 : 0.0580, 재현율:0.5800,F1 스코어:0.1055
ROC 스코어: 0.6883, PR score : 0.0425
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1931  384]
 [  22   28]]
정확도: 0.8283, 정밀도 : 0.0680, 재현율:0.5600,F1 스코어:0.1212
ROC 스코어: 0.6971, PR score : 0.0474
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1931  384]
 [  22   28]]
정확도: 0.8283, 정밀도 : 0.0680, 재현율:0.5600,F1 스코어:0.1212
ROC 스코어: 0.6971, PR score : 0.0474
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2039  276]
 [  26   24]]
정확도: 0.8723, 정밀도 : 0.0800, 재현율:0.4800,F1 스코어:0.1371
ROC 스코어: 0.6804, PR score : 0.0494
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2039  276]
 [  26   24]]
정확도: 0.8723, 정밀도 : 0.0800, 재현율:0.4800,F1 스코어:0.1371
ROC 스코어: 0.6804, PR score : 0.0494
-

In [26]:
# knn 파라미터 구하기
from sklearn.metrics import accuracy_score
acc = []
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    for k in range(1,20):
        knn_clf = KNeighborsClassifier(n_neighbors=k)
        knn_clf.fit(x_train, y_train)
        knn_pred = knn_clf.predict(x_test)
        knn_score=accuracy_score(y_test,knn_pred)

        acc.append(knn_score)
    int = acc.index(max(acc))+1
    print("최적거리")
    print(int)

    knn_clf_best =KNeighborsClassifier(n_neighbors=int)
    knn_clf_best.fit(x_train, y_train)
    knn_best_pred = knn_clf_best.predict(x_test)
    knn_best_pred_proba = knn_clf_best.predict_proba(x_test)   

    print(feature,"KNN 파라미터적용") 
    get_clf_eval(y_test, knn_best_pred)
    get_eval_by_threshold(y_test ,knn_best_pred_proba[:,1].reshape(-1,1), thresholds)      


최적거리
2
mda KNN 파라미터적용
오차행렬
[[2172  143]
 [  35   15]]
정확도: 0.9247, 정밀도 : 0.0949, 재현율:0.3000,F1 스코어:0.1442
ROC 스코어: 0.6191, PR score : 0.0433
임곗값: 0.1
오차행렬
[[2051  264]
 [  26   24]]
정확도: 0.8774, 정밀도 : 0.0833, 재현율:0.4800,F1 스코어:0.1420
ROC 스코어: 0.6830, PR score : 0.0510
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2051  264]
 [  26   24]]
정확도: 0.8774, 정밀도 : 0.0833, 재현율:0.4800,F1 스코어:0.1420
ROC 스코어: 0.6830, PR score : 0.0510
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2051  264]
 [  26   24]]
정확도: 0.8774, 정밀도 : 0.0833, 재현율:0.4800,F1 스코어:0.1420
ROC 스코어: 0.6830, PR score : 0.0510
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2051  264]
 [  26   24]]
정확도: 0.8774, 정밀도 : 0.0833, 재현율:0.4800,F1 스코어:0.1420
ROC 스코어: 0.6830, PR score : 0.0510
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2172  143]
 [  35   15]]
정확도: 0.9247, 정밀도 : 0.0949, 재현율:0.3000,F1 스코어:0.1442
ROC 스코어: 0.6191, PR sc

In [27]:
# 1-5 xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 

xgb_wrapper = XGBClassifier()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    xgb_wrapper.fit(x_train, y_train, verbose = True)
    xgb_wrapper.fit(x_train, y_train)
    xgb_pred = xgb_wrapper.predict(x_test)
    xgb_pred_proba = xgb_wrapper.predict_proba(x_test)
    print(feature,"XGB") 
    get_clf_eval(y_test, xgb_pred)
    get_eval_by_threshold(y_test , xgb_pred_proba[:,1].reshape(-1,1), thresholds)

mda XGB
오차행렬
[[2214  101]
 [  33   17]]
정확도: 0.9433, 정밀도 : 0.1441, 재현율:0.3400,F1 스코어:0.2024
ROC 스코어: 0.6482, PR score : 0.0629
임곗값: 0.1
오차행렬
[[2017  298]
 [  26   24]]
정확도: 0.8630, 정밀도 : 0.0745, 재현율:0.4800,F1 스코어:0.1290
ROC 스코어: 0.6756, PR score : 0.0468
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2115  200]
 [  29   21]]
정확도: 0.9032, 정밀도 : 0.0950, 재현율:0.4200,F1 스코어:0.1550
ROC 스코어: 0.6668, PR score : 0.0522
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2159  156]
 [  31   19]]
정확도: 0.9209, 정밀도 : 0.1086, 재현율:0.3800,F1 스코어:0.1689
ROC 스코어: 0.6563, PR score : 0.0544
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2191  124]
 [  32   18]]
정확도: 0.9340, 정밀도 : 0.1268, 재현율:0.3600,F1 스코어:0.1875
ROC 스코어: 0.6532, PR score : 0.0592
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2214  101]
 [  33   17]]
정확도: 0.9433, 정밀도 : 0.1441, 재현율:0.3400,F1 스코어:0.2024
ROC 스코어: 0.6482, PR score : 0.0629
-

In [28]:
values[1]

'l2'

In [29]:
#xgb 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

xgb_wrapper
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {
            'n_estimators':[40,43,46,50],
            'learning_rate':[0.03,0.05,0.04],
            'max_depth' : [0.8,1,2,3,4]
            }

    grid_clf = GridSearchCV(xgb_wrapper, param_grid=params, scoring='accuracy', cv=5)
    grid_clf.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

    a = grid_clf.best_params_
    keys=list(a.keys())
    values = list(a.values())
    xgb_clf_best = XGBClassifier( learning_rate=values[0],max_depth=values[1],n_estimators= values[2])
    xgb_clf_best.fit(x_train, y_train)
    xgb_best_pred = xgb_clf_best.predict(x_test)
    xgb_best_pred_proba = xgb_clf_best.predict_proba(x_test)   

    print(feature,"XGB 파라미터적용") 
    get_clf_eval(y_test, xgb_best_pred)
    get_eval_by_threshold(y_test ,xgb_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미어:{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 50}, 최적 평균 정확도:0.860
mda XGB 파라미터적용
오차행렬
[[1925  390]
 [  15   35]]
정확도: 0.8288, 정밀도 : 0.0824, 재현율:0.7000,F1 스코어:0.1474
ROC 스코어: 0.7658, PR score : 0.0640
임곗값: 0.1
오차행렬
[[ 316 1999]
 [   3   47]]
정확도: 0.1535, 정밀도 : 0.0230, 재현율:0.9400,F1 스코어:0.0448
ROC 스코어: 0.5383, PR score : 0.0229
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1210 1105]
 [   8   42]]
정확도: 0.5294, 정밀도 : 0.0366, 재현율:0.8400,F1 스코어:0.0702
ROC 스코어: 0.6813, PR score : 0.0341
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1671  644]
 [  11   39]]
정확도: 0.7230, 정밀도 : 0.0571, 재현율:0.7800,F1 스코어:0.1064
ROC 스코어: 0.7509, PR score : 0.0492
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1819  496]
 [  13   37]]
정확도: 0.7848, 정밀도 : 0.0694, 재현율:0.7400,F1 스코어:0.1269
ROC 스코어: 0.7629, PR score : 0.0569
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1925  390]
 [  15